### CS4423 - Networks
Angela Carnevale  
School of Mathematical and Statistical Sciences  
University of Galway

#### 4. Random Networks

# Week 7, lecture 1: Erdős-Rényi Random Graph Models.

A [random graph](https://en.wikipedia.org/wiki/Random_graph) is a **mathematical model of a family of networks**, where certain parameters (like the number of nodes and edges) have fixed values, but other aspects (like the actual edges) are randomly assigned. 

The simplest example of a random graph is in fact a network with fixed numbers $n$ of nodes and $m$ of edges,  randomly placed between the vertices. Although a random graph is not a specific object, many of its properties can be described precisely in the form of **expected values** or **probability distributions**.

<img src="https://upload.wikimedia.org/wikipedia/commons/thumb/1/13/Erdos_generated_network-p0.01.jpg/1280px-Erdos_generated_network-p0.01.jpg">

In [ ]:
import networkx as nx
import random
opts = { "with_labels": True, "node_color": 'y'}
opts1 = { "with_labels": True, "node_color": 'lightblue'}

## Random Samples

* Our goal is to randomly select edges on a given vertex set $X$. That is, pick at random elements from the set $\binom{X}{2}$ of pairs of nodes.

* So we need a procedure
for selecting $m$ from $N$ objects randomly, in such a way that each of the $\binom{N}{m}$
subsets of the $N$ objects is an equally likely outcome.

* We first discuss sampling $m$ values in the range $\{0, 1, \dots, N{-}1\}$.

### An intuitive approach.

Maybe the most obvious approach is to select each number in the desired range with probability $p = m/N$.

* `Python`'s basic random number generator `random.random` returns a random number in the (half-open) interval $[0, 1)$ every time it is called.  

* Looping with `a` over `range(N)`: if the randomly generated number is less than $p$, then we include the current value of `a`, if not we don't.

In [ ]:
def random_sample_B(N, p):
    "sample elements in range(n) with probability p"
    sample = []
    for a in range(N):
        if random.random() < p:
            sample.append(a)
    return sample 

Let's see a small example:

In [ ]:
N = 100
m = 10

In [ ]:
pick = random_sample_B(N, m/N)
pick

In [ ]:
len(pick)

**Note.** Unfortunately, this gives a sample of size $m$ only **on average**.
Let's check the average size of, say, $10000$ samples.

In [ ]:
c = 10000
sum(len(random_sample_B(N, m/N)) for i in range(c))/c

Let $k$ be the sample size, as a random variable. Then its

**Expected value:** $E[k] = Np = m$.

**Standard deviation:** $\sigma_k=\sqrt{m(1-m/N)}$. 

**Proof:** This is a [binomial distribution](https://en.wikipedia.org/wiki/Binomial_distribution).

* The probability of a specific subset of size $k$ to be chosen is $p^k (1-p)^{N-k}$.

* As there are $\binom{N}{k}$ subsets of size $k$, the probabilty $P(k)$
  of the sample to have size $k$ is $P(k) = \binom{N}{k} p^k (1-p)^{N-k}$.

**Expected value:**

* Thus $E[k] = \sum_{j=0}^N P(j) \cdot j = \sum_{j=0}^N j \binom{N}{j} p^j (1-p)^{N-j} 
 = Np\sum_{l=0}^{N-1} \binom{N-1}{l} p^l (1-p)^{(N-1)-l}
= Np
$, substituting $l = k - 1$, 


* as $j \binom{N}{j} p^j = Np \binom{N-1}{j-1} p^{j-1}$, $(1-p)^{N-j} = (1-p)^{(N-1)-(j-1)}$ and $(p+(1-p))^r=1$ for all $r$.

**Standard deviation:** 

One can check that variance and standard deviation are as follows:


* $E[k^2] - E[k]^2 = N(N-1)p^2 + Np - (Np)^2 = Np(1-p)$.


* Thus $\sigma_k = \sqrt{m(1-m/N})$.

In [ ]:
c = 10000
sum(1 for i in range(c) if len(random_sample_B(N, m/N)) == m)

### Another strategy.

A simple modification of this procedure will always yield samples of size $m$
[see Knuth: The Art of Computer Programming, Vol. 2, Section 3.4.2, Algorithm S] :

* The number $a$ should be selected with probability $(m-c)/(N-a)$,
if $c$ items have already been selected.

In [ ]:
def random_sample_A(N, m):
    sample = []
    for a in range(N):
        if (N - a) * random.random() < m - len(sample):
            sample.append(a)
    return sample

Let's see a small example.

In [ ]:
N = 100
m = 10
pick = random_sample_A(N, m)
pick

In [ ]:
len(pick)

We now apply these strategies A and B to the problem of randomly selecting edges for a graph on the vertex set $X = \{0, 1, \dots, n-1\}$.

## Uniformly Selected Edges

* Let us denote by $G(n, m)$ a network with $n$ nodes and $m$ chosen
edges, chosen uniformly at random (out of the possible
$\binom{n}{2}$).  

* Equivalently, one can choose uniformly at random
one network in the **set** $G(n, m)$ of **all** networks on a given set
of $n$ nodes with **exactly** $m$ edges.

**Definition (ER Model $A$: Uniform Random Graphs).**

Let $n \geq 1$, let $N = \binom{n}{2}$ and let $0 \leq m \leq N$.

The model $G(n, m)$ consists of the ensemble of graphs $G$
with $n$ nodes $X = \{0, 1, \dots, n{-}1\}$, and $m$ randomly selected
edges, chosen uniformly from the $N=\binom{n}{2}$ possible edges.

One could think of $G(n, m)$ as a probability distribution
$P \colon G(n, m) \to \mathbb{R}$,
that assigns to each network $G \in G(n, m)$ the same probability
$$
P(G) = \binom{N}{m}^{-1},
$$
where $N = \binom{n}{2}$.

For example ... (Picture from [_Complex Networks_, Latora et al.])

<img src="images/random-gnm.png">

* A function `random_graph_A` that
takes the order $n$ and the size $m$ of a random ER graph of type A
as arguments, and constructs such a graph can simply follow the 
`random_sample_A` algorithm and sample exactly $m$ of the $N = \binom{n}{2}$
available edges.  

* We introduce explicit counters
`a` (for the number of edges that have been considered)
and `c` (for the number of chosen edges).

In [ ]:
def random_graph_A(n, m):
    """construct a random type A graph
    with n nodes and m links"""
    a = c = 0
    N = n * (n-1) // 2 ## no. possible edges for graph with n nodes
    G = nx.empty_graph(n)
    for x in range(n):
        for y in range(x):
            if (N - a) * random.random() < m - c:
                G.add_edge(x, y)
                c += 1
            a += 1
    return G

Now we can construct and draw a random graph on $16$ vertices with $15$ edges.

In [ ]:
n = 16
m = 15

In [ ]:
G = random_graph_A(n, m)
nx.draw(G, **opts)

In [ ]:
G.number_of_edges()

The `networkx` version of this random graph constructor is called `gnm_random_graph`
and should produce the same random graphs with the same probability. 

In [ ]:
G = nx.gnm_random_graph(n, m)
nx.draw(G, **opts1)

In [ ]:
list(G.nodes())

## Randomly Selected Edges

**Definition (ER Model $B$: Binomial Random Graphs).**
Let $n \geq 1$, let $N = \binom{n}{2}$ and let $0 \leq p \leq 1$.


The model $G(n, p)$ consists of the ensemble of graphs $G$
with $n$ nodes $X = \{0, 1, \dots, n{-}1\}$, and each of the $N$
possible edges chosen with probability $p$.

The probability $P_G$ of a particular graph $G = (X, E)$ with 
$X = \{0, 1, \dots, n{-}1\}$ and
$m = |E|$ edges in the $G(n, p)$ model is
$$
P_G = p^m(1-p)^{N-m}.
$$

For example ... (Picture from [_Complex Networks_, Latora et al.])

<img src="images/random-gnp.png">

Such a random graph is easy to generate programmatically, 
following the `random_sample_B` algorithm.

* `python`'s basic random number generator
`random.random` returns a random number in the (half-open) interval $[0, 1)$,
every time it is called.  

* Looping over the possible edges, we generate a random number this way. If this number is less than $p$, we include the edge, if not we don't.

In [ ]:
def random_graph_B(n, p):
    """construct a random type B graph
    with n nodes and edge probability p"""
    G = nx.empty_graph(n)
    for x in range(n):
        for y in range(x):
            if random.random() < p:
                G.add_edge(x, y)
    return G

In [ ]:
n = 16
m = 15
N = n*(n-1)//2
m/N

In [ ]:
G = random_graph_B(n, m/N)
nx.draw(G, **opts)
G.size()

In [ ]:
G.number_of_nodes(), G.number_of_edges()

The `networkx` version of this random graph constructor is called `gnp_random_graph`
and should produce the same random graphs with the same probability. 

In [ ]:
G = nx.gnp_random_graph(n, m/N)
nx.draw(G, **opts1)

In [ ]:
G.number_of_nodes(), G.number_of_edges()

##  Code Corner

### `random`

* `random`: [[doc]](https://docs.python.org/2/library/random.html#random.random)


* `sample`: [[doc]](https://docs.python.org/2/library/random.html#random.sample)

### `numpy`

* `random.choice`: [[doc]](https://docs.scipy.org/doc/numpy-1.14.0/reference/generated/numpy.random.choice.html)

### `networkx`

* `gnm_random_graph`: [[doc]](https://networkx.github.io/documentation/stable/reference/generated/networkx.generators.random_graphs.gnm_random_graph.html)


* `gnp_random_graph`: [[doc]](https://networkx.github.io/documentation/stable/reference/generated/networkx.generators.random_graphs.gnp_random_graph.html)

##  Exercises

1. Why does the algorithm used in the `random_sample_A` function above pick every $m$-element subset
of the set $\{0, 1, \dots, N-1\}$ with the same probability?

1. Design an experiment that verifies this claim.  (E.g., there are $\binom{10}{3} = 120$ different $3$-element
subsets in a $10$-element set.  One could use `python`'s `frozenset`s to make each of these $3$-element sets
a key in a dictionary which counts how often each set is picked when `random_sample_A` is applied a large number of times ...)

1.  What is the expected number $E[m]$ of edges in the $G(n, p)$ model of random graphs?
